In [ ]:
!pip uninstall -y tensorflow  # Remove TensorFlow
!pip install torch  # Install PyTorch
!pip install python-doctr[torch]  # Install docTR with PyTorch backend

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 820.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.5/299.5 kB 16.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=0a7ee8e74465823b342a44d329a753f326a99bd64d5208a4e4d01ea9c1e796bf
  Stored in directory: /root/.cache/

In [ ]:
pip install python-doctr[torch]

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/Sample_test.csv')

# Assuming the CSV has a column 'file_path' with image paths
file_paths = df['image_link'].tolist()

In [ ]:
import requests
import pandas as pd
import os
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

# Initialize OCR model with vertical text handling
model = ocr_predictor(pretrained=True, assume_straight_pages=False, export_as_straight_boxes=True)

# Create a directory for downloaded images
image_dir = 'downloaded_images'
os.makedirs(image_dir, exist_ok=True)

# Load the CSV with URLs
df = pd.read_csv('/content/Sample_test.csv')

# List to hold results
results = []

# Function to extract text from the OCR result
def extract_text(ocr_result):
    extracted_text = []
    # Traverse the result and extract the 'value' fields (words)
    for page in ocr_result['pages']:
        for block in page['blocks']:
            for line in block['lines']:
                for word in line['words']:
                    extracted_text.append(word['value'])
    # Join all words with a space
    return ' '.join(extracted_text)

# Iterate over each URL in the CSV
for idx, row in df.iterrows():
    image_url = row['image_link']  # Adjust column name if necessary

    try:
        # Download the image
        response = requests.get(image_url, stream=True)
        if response.status_code == 200:
            # Create the local file path
            image_filename = os.path.join(image_dir, f"image_{idx}.jpg")
            with open(image_filename, 'wb') as img_file:
                img_file.write(response.content)

            # Perform OCR on the downloaded image
            doc = DocumentFile.from_images(image_filename)
            result = model(doc)
            extracted_text = extract_text(result.export())  # Extract only the text values
            results.append({'file_path': image_url, 'extracted_text': extracted_text})
        else:
            print(f"Failed to download {image_url}, status code: {response.status_code}")
    except Exception as e:
        print(f"Error processing {image_url}: {e}")

# Save the results to a CSV
output_df = pd.DataFrame(results)
output_df.to_csv('generated_text.csv', index=False)

print("OCR process completed. Results saved to 'generated_text.csv'.")

OCR process completed. Results saved to 'generated_text.csv'.
